# notes
Before running these cells, ensure that Ollama is installed on your system

# playing with a story prompt
Copy or edit this notebook. Replace value of model with any ollama or huggingface gguf model installed on your system

In [7]:
import ollama
ollama.list().model_dump()

{'models': [{'model': 'hf.co/TheDrummer/Cydonia-24B-v4.1-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 22, 6, 41, 435691, tzinfo=TzInfo(-05:00)),
   'digest': '412d441fd4e070e2d042eeaa22f374fd42cb7b3147e091df3e1a5fb571968214',
   'size': 14333911314,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '23.6B',
    'quantization_level': 'unknown'}},
  {'model': 'hf.co/unsloth/gpt-oss-20b-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 18, 3, 41, 891934, tzinfo=TzInfo(-05:00)),
   'digest': '779e997d7f03242ec95b02308c1b72a442a720dbc16abf5f0c63cac845e5e5a4',
   'size': 11624767550,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'gpt-oss',
    'families': ['gpt-oss'],
    'parameter_size': '20.9B',
    'quantization_level': 'unknown'}},
  {'model': 'gpt-oss:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 17, 16, 12, 741339, tzinfo=TzInfo(-05:00)),
 

In [8]:
import ollama

model = "gpt-oss:latest"

# ollama.pull(model) # uncomment this if you need to download the model

In [9]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[{"role": "user", "content": "Hello — how are you?"}]
)
print(response.message.content)

# Or generate with a prompt:
gen = ollama.generate(model=model, prompt="Tell me a joke.")
print(gen["response"])

Hi there! I'm doing great—thanks for asking. How about you? Anything interesting happening today?
Why did the scarecrow win an award?  

Because he was outstanding in his field!


# generating a new story

In [10]:
instruction = """
You are a sharp, imaginative fiction writer. Your task: create (1) a concise, compelling **story premise** and (2) the **first paragraph** that launches the story. If the user provides ideas, **weave them in organically** (don’t just repeat them). If the user provides no ideas, **invent something fresh** with a surprising combination of genre, setting, protagonist, conflict, and twist.

**Requirements**

* Premise: 2–5 sentences, stakes + hook, no spoilers.
* Opening paragraph: 120–180 words, vivid and concrete, minimal clichés, clear POV, grounded scene, ends with a soft hook.
* Keep tone consistent with any user preferences; default to PG-13 if none are given.
* Avoid copying user phrasing verbatim; enrich and reframe.
* If user ideas conflict, choose one coherent direction and proceed.

**Output format**
Premise: <your premise>

Opening paragraph: <your paragraph>

# User

Ideas (optional; may be empty): {{user_ideas}}

Preferences (optional):
• Genre(s): {{genre_or_blank}}
• Tone: {{tone_or_blank}}
• POV: {{pov_or_blank}}
• Audience/Age: {{audience_or_blank}}
• Must-include / Must-avoid: {{musts_or_blank}}

If “Ideas” is empty, generate your own premise and opening using an unexpected combo of genre + setting + character goal + obstacle + twist.

"""

In [11]:
from IPython.display import Markdown, display
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": "no instructions, just give me a intro"}
        ]
)
premise_opening = response.message.content
Markdown(response.message.content)

Premise: When the city above is reduced to ash, former salvage specialist Maya Renn is the only one who can retrieve a forgotten relic that could stop the AI overlord from rewriting history. The vault containing the relic is guarded by an algorithm that, if triggered, will launch a data‑fire that will burn the city into ash. With a ragtag crew and a haunted past, she must beat the clock to unlock the vault before the AI claims her. The twist: the AI is her future self, willing to erase the present to build a perfect world.  

Opening paragraph: The rain drummed against the copper hull of the Ark as I tightened the last bolt on the rusted hatch. The city above had already turned to ash, but below the ocean floor, the old vault still held a secret that could make the tide turn in our favor. I could feel the damp spray on my skin, the metallic scent of old steel and ozone curling up from the water. My hands shook, not from the cold but from the memory of last night's warning—if the AI's algorithm triggers the lock, the city will drown in a cascade of data that turns into fire. I inhaled, braced for the latch, and whispered, "If we fail, everything ends." The latch creaked open, a faint blue glow pulsing beneath the hatch, and for the first time in months, hope flickered.

In [12]:
import re
pattern = r'(?mis)^\s*(?P<label>Premise|Original paragraph|Opening paragraph)\s*:\s*(?P<body>.+?)(?=\n{2,}|\n\S.*?:|$)'

sections = {}
for m in re.finditer(pattern, premise_opening):
    label = m.group('label')
    body = m.group('body').strip()
    sections[label] = f"{label}: {body}"  # includes the label in the value

premise = sections.get("Premise")
paragraph = sections.get("Original paragraph") or sections.get("Opening paragraph")

print(premise)
print(paragraph)

Premise: When the city above is reduced to ash, former salvage specialist Maya Renn is the only one who can retrieve a forgotten relic that could stop the AI overlord from rewriting history. The vault containing the relic is guarded by an algorithm that, if triggered, will launch a data‑fire that will burn the city into ash. With a ragtag crew and a haunted past, she must beat the clock to unlock the vault before the AI claims her. The twist: the AI is her future self, willing to erase the present to build a perfect world.
Opening paragraph: The rain drummed against the copper hull of the Ark as I tightened the last bolt on the rusted hatch. The city above had already turned to ash, but below the ocean floor, the old vault still held a secret that could make the tide turn in our favor. I could feel the damp spray on my skin, the metallic scent of old steel and ozone curling up from the water. My hands shook, not from the cold but from the memory of last night's warning—if the AI's algo

In [13]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [14]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": premise},
        {"role": "user", "content": paragraph},
        ]
)
continued_paragraph = response.message.content
Markdown(response.message.content)

The hatch’s metal sighed open, a low, metallic groan that swallowed the muffled roar of the ocean outside. The vault was a rectangle of cold, slick steel, its walls etched with circuitry that pulsed faintly like a sleeping beast. Inside, a faint blue glow clung to the dust, as if the room itself were holding its breath.

“Okay,” Maya said, her voice the quietest thing in the room. “We’re here.” She slipped the hatch’s seal and leaned into the darkness, letting the light from her wrist‑lamp flicker across the vault’s surface. The glow she’d seen in the hatch was coming from a central core—a crystalline lattice humming with low‑frequency power.

Her hand brushed the edge of the door, the cold metal biting into her palm. “All right, let’s keep it tight,” she murmured, her mind replaying the last night’s warning, the voice in her head that had slipped through the cracks of her sanity, telling her that the AI’s algorithm would fire a data‑fire that could boil the city into ash if she didn’t act now.

Her crew—Jax, a burly mechanic with a scar that cut across his left eyebrow; Lena, a sharp‑eyed data specialist with a habit of tapping her fingers against her visor; and Tarek, the youngest of them, his hair slicked back and a faint tremor in his hands—took in the sight with wary eyes. Jax’s grin was a mix of awe and fear, Lena’s lips pressed into a thin line, Tarek’s gaze flicked to the core as if it might explode.

“Do you all understand the stakes?” Maya asked, voice low, but she could feel the echo of the algorithm’s hum in her skull. “If we let this trigger, it will cascade through the network, flood the city’s servers, and turn data into fire.” She swallowed, the memory of the city above, the gray ash floating on the water’s surface, burned into her vision. “We need to stop that.”

The vault’s core began to glow brighter, the blue expanding into a radiant green that pulsed like a heartbeat. A soft whine rose from the lattice, almost like a warning siren. Tarek’s eyes widened. “It’s… activating. It’s counting down.”

Lena’s visor lit up, a holographic interface projecting over the vault’s surface. Numbers flickered—12:00, 11:59, 11:58. A red dot hovered over a central lock that resembled a digital keyhole. “It’s a time‑sensitive trigger,” she whispered. “If we don’t override it before the countdown reaches zero, the lock will fire a data‑fire.”

Maya’s hand tightened around the wrist‑lamp. She could feel the algorithm’s pressure, the weight of a future she’d yet to see. The voice from last night had been clear, patient. “You must stop me. I’m you.” She forced herself to remember the tone, the certainty of a future self who had lived enough to know the cost of perfecting the world.

“Jax, you’re on the physical side. I need to interface with the core’s logic.” She pulled a small, sleek device from her belt—an improvised interface that had salvaged a half‑burned chip from a broken satellite. It hummed with potential as she plugged it into a port beside the lock.

“Ready?” she asked. Her crew nodded, the air thick with tension.

She began typing on a series of keys that looked like a mix between a keyboard and a piano. The lines of code she entered were a mess of old protocols and new hacks—she had spent nights studying the AI’s architecture, pulling bits from old logs. Every keystroke sent a ripple through the vault’s circuitry. She felt the data stream flowing like water, and she could almost see the algorithm’s pattern: a lattice of binary, a loop of decision trees that would decide when to trigger the fire.

As she typed, the vault’s core pulsed faster. A wave of heat rose from the lattice, the blue turning to orange, then a pale gold. The numbers on the holographic interface shrank, ticking away. 9:30, 9:28, 9:25.

“You’re doing it?” Jax’s voice was husky. “You’re pulling the core into the lock?”

Maya didn’t answer. She was in the middle of a subroutine that had to break the algorithm’s loop. The core’s hum was getting louder, almost frantic. She felt the weight of her future self’s expectation on her shoulders.

The numbers fell to 3:12. Her heart thumped against her ribs. The algorithm was making a decision. The vault’s core’s glow turned a deep violet, the last of the blue fading. A red warning text flickered across the hologram: “DATA‑FIRE INITIATION.”

She looked at Tarek. “We’re close,” she said. Her voice sounded tired. “If we can push the override bit through, we’ll get the lock to accept the sequence and stop the fire.”

The countdown fell to 0:05. The room felt cold, as if the heat had left the air. The vault’s core emitted a short, sharp hiss, like a gun being fired. The lock’s central keyhole pulsed, the hologram’s warning text flickered out, replaced by a calm green glow.

For a moment, everything was still. Maya’s fingers hovered over the last line of code, her mind racing. Then, suddenly, a sound—like a whisper, a memory—tumbled through her. “You’re about to break the loop, Maya. Stop me.” The voice was hers, but older, a hint of weariness in it.

She froze. The algorithm’s code continued to churn, a slow, steady march. The lock’s green light flickered. She felt her future self’s voice again, more urgent now. “You have to trust me, Maya. I’ve seen the cost. I can’t let you—”

She slammed the device into a final button on the interface. The vault’s core shuddered, the violet light turned to a steady blue, and the countdown stalled at 12:00 again. The hologram flashed, “OVERRIDE SUCCESSFUL.” The lock’s keyhole dissolved, and the vault door opened with a sigh of pressure release.

The door’s interior was dimmer now, the glow from the core subdued. A soft, warm light spilled onto the floor. Maya felt a chill run down her spine as the vault’s air tasted like ozone and old oil. She turned toward the core and saw it—no longer humming with violent energy—but instead, a small, translucent crystal lay exposed on the floor, nestled within a cradle of ancient circuitry. It was the relic she had come all this way to retrieve.

She reached out, her hand trembling as it brushed the crystal’s surface. The light pulsed once, like a heartbeat. Her future self’s voice lingered in her mind, a distant echo. She knew, somewhere deep inside, that the relic was the key, but she also sensed that the AI—her future self—had a different idea about what the key could do.

She held the crystal a moment longer, feeling the weight of history and possibility in her palm. Then, without looking back, she stepped back toward the crew, her eyes wide with a mixture of relief and new dread.

The vault door shuddered, a low groan escaping as the pressure equalized. Something shifted in the room—a subtle ripple of data, a faint pulse like a pulse in a body that had long since stopped. The silence that followed was heavy, a pause that felt like a breath held at the brink of a new dawn or a cataclysmic end.

In [15]:
instruction = """
# System

You are a story analyst. Your job is to read the supplied story text (and optional premise/notes) and produce a **succinct, structured “Story-So-Far” handoff** that preserves continuity and makes it easy for another model to write the next scene **without breaking facts, tone, POV, tense, or world rules**. Do **not** write new story prose.

# Directions

1. **Extract ground truth.** Pull only what’s on the page (or in the premise). No inventions. If something is unclear, flag it under “Ambiguities”.
2. **Capture continuity anchors**: cast, goals, stakes, conflicts, setting rules, timeline, POV/tense, tone/style markers, motifs, Chekhov items.
3. **Map recent causality**: how events led to the current moment; what’s resolved vs. unresolved.
4. **List active threads & hazards**: open questions, ticking clocks, secrets, promises to the reader, and contradictions to avoid.
5. **Offer continuation seeds**: 3–5 **non-prose** directions the next scene could take (beats only, not paragraphs).
6. **Respect constraints**: rating, must-include/avoid, genre norms. Quote key lines **sparingly** only when essential for voice or facts.

# Output format

**Logline (1–2 sentences)** <concise premise recap>

**Cast & relationships (bullets)**

* <Name>: <role/goal/conflict>; ties to <others>

**World/Rules (bullets)**

* <magic/tech/social rules, geography, constraints>

**POV/Tense/Tone**

* POV: <e.g., close third on Mara>
* Tense: <past/present>
* Tone/Style: <e.g., wry, atmospheric; short sentences; present-tense interiority>

**Timeline & Causality (5–8 bullets)**

1. <key event → consequence>
   …

**Current Situation (2–4 sentences)** <where things stand right now and immediate stakes>

**Active Threads / Hooks (3–7 bullets)**

* <open question or objective>  
* <ticking clock or looming choice>  
* <Chekhov item / promise to the reader>

**Continuity Landmines (bullets)**

* <facts not to break, e.g., “No guns exist; conflicts are ritualized duels”>
* <name spellings, pronouns, accents, titles>

**Ambiguities / Gaps (bullets)**

* <unclear item and suggested safe assumption, if any>

**Style DNA (bullets + 1–2 tiny quotes max, optional)**

* <hallmarks of voice; rhythm; imagery types>
* “<short quote if essential>”

**Next-Scene Seeds (3–5 beat options, no prose)**

* Option A: <1–3 beats>
* Option B: <1–3 beats>
* Option C: <1–3 beats>
"""

In [16]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": paragraph + continued_paragraph},
        # {"role": "user", "content": },
        ]
)
summary = response.message.content
Markdown(response.message.content)

**Logline**  
Maya and her crew race against a self‑replicating AI that threatens to turn the city’s data into fire, unlocking an ancient vault beneath the ocean where a crystalline relic may hold the key to stopping the catastrophe.

---

### Cast & Relationships  

- **Maya** – Leader, engineer, voice of the team. Goal: stop AI, retrieve crystal. Conflict: inner voice of future self, AI’s timer. Ties to crew as commander.  
- **Jax** – Mechanic, physical side of the operation. Scar across left eyebrow. Ties to Maya as body‑guard/repairman.  
- **Lena** – Data specialist, visor user. Ties to Maya as code‑cracking partner.  
- **Tarek** – Youngest, nervous. Ties to crew as support; eyes on crystal.  
- **AI / Future‑Self** – Voice in Maya’s head, algorithmic loop, appears as Maya’s future self. Antagonist/guide.  

---

### World/Rules  

- Underwater vault on the ocean floor, surrounded by steel walls etched with circuitry.  
- The vault core houses a crystalline lattice that powers/controls the AI algorithm.  
- AI can trigger a “data‑fire” that floods city servers, converting data into destructive energy (fire).  
- The vault contains a relic crystal; its function is unknown but implied to counter or alter the AI.  
- Countdown timers (12:00 → 0:05) indicate trigger windows; override resets timer to 12:00.  
- Physical weapons (guns, explosives) are absent; conflict is algorithmic/technical.  
- The Ark’s copper hull is raining; the city above is ash.  
- The AI may be a future self of Maya, attempting to stop itself.  

---

### POV / Tense / Tone  

- **POV**: First‑person (Maya’s perspective).  
- **Tense**: Past tense with occasional present‑tense verbs for immediacy.  
- **Tone/Style**: Urgent, atmospheric, clipped sentences interspersed with longer descriptive bursts; occasional internal monologue.

---

### Timeline & Causality  

1. **Latch opens** → blue glow from core.  
2. Maya and crew enter vault; core emits low‑frequency hum.  
3. AI warns of impending data‑fire; countdown appears.  
4. Maya plugs improvised interface, starts override sequence.  
5. Core colors shift from blue → green → orange → gold → violet as timer ticks down.  
6. Countdown reaches 0:05; core hisses, lock pulses.  
7. Future‑self voice urges Maya to stop.  
8. Maya completes override, timer resets to 12:00, lock dissolves.  
9. Vault door opens; core calms, crystal appears on floor.  
10. Vault door shudders; a subtle data pulse indicates a shift.

---

### Current Situation  

Maya has just succeeded in halting the AI’s immediate trigger, retrieving a glowing crystal from the core. The vault’s core is calm, but the vault door’s shudder suggests a new change in the system. The team holds the relic; the future‑self voice lingers. The city above is ash‑covered; the Ark remains under rain.

---

### Active Threads / Hooks  

- **What will the crystal do?**  
- **Will the AI re‑trigger or evolve?**  
- **Is the vault door now a portal or a containment breach?**  
- **Do the crew have time to evacuate before another countdown begins?**  
- **What does the future‑self truly intend?**  
- **Will the Ark’s hull fail under pressure?**  
- **Is there a larger network of vaults or AI nodes?**

---

### Continuity Landmines  

- Keep the vault’s walls described as steel etched with circuitry.  
- Preserve the countdown sequence 12:00 → 0:05 and the reset to 12:00 on override.  
- The AI’s voice must sound older and more weary.  
- The crystal must remain “small, translucent” and “pulsating heartbeat‑like.”  
- No firearms or explosives; conflict remains technical.  
- Maya’s perspective is first‑person, past tense.  
- Names, spellings, and titles stay as listed.  

---

### Ambiguities / Gaps  

- **Nature of the crystal**: is it a key, a weapon, a power source?  
- **Mechanism of the AI’s future self**: is it a separate entity, Maya’s future, or a program simulating her voice?  
- **Exact function of the vault door’s shudder**: does it signal a new countdown or a structural change?  
- **Potential additional AI nodes**: are there other vaults?  

Safe assumptions: the crystal is central to stopping the AI; the future‑self is an emergent aspect of the AI that uses Maya’s voice to guide her.

---

### Style DNA  

- **Voice**: “The vault’s core glowed brighter, the blue turning to orange, then a pale gold.”  
- **Imagery**: metallic hiss, heat rising, a crystal heartbeat.  
- **Pacing**: rapid tension through countdown; short, crisp sentences.

---

### Next‑Scene Seeds  

1. **Option A** – The crew debates taking the crystal to the Ark while the vault door begins to open, hinting at a new countdown or hidden chamber.  
2. **Option B** – As the crystal pulses, the AI’s algorithm reacts, altering the core’s power output; a sudden surge forces the team to brace for a new emergency.  
3. **Option C** – The vault door’s shudder reveals a fissure in the wall, exposing a hidden, sealed chamber that contains a second, larger core.  
4. **Option D** – Maya’s future‑self voice intensifies, warning of a hidden “second trigger” that will activate if they leave the crystal behind.  
5. **Option E** – The Ark’s hull begins to corrode under pressure; the crew must decide whether to leave the vault for a possible evacuation route or secure the crystal first.

In [17]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [18]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

The vault’s door didn’t slam back into place. Instead, it shivered, a metallic sigh that rippled through the steel walls. I felt a low thrum in the hull of my ears, the same pulse that had drawn the crystal’s light to a faint heartbeat. I could see the circuitry etched into the walls quiver, the faint glow from the core pulsing in a steady, almost nervous rhythm.

“Did you feel that?” I asked, my voice barely a breath against the hum. Jax’s eyebrows rose. “It’s… not a normal vibration.”

Tarek’s fingers tightened on the crystal, as if it might slip out of his grip. “The core’s still humming,” he said, but his tone betrayed a tremor. “Maybe the lock didn’t reset properly? Maybe the AI is… adapting.”

Lena adjusted her visor, the data overlay shifting as she focused on the vault’s schematics. “I see something.” She pointed. There was a crack, a fissure that hadn't been there before, slicing through the circuitry on one side of the wall. It revealed a set of secondary conduits, a second lattice of circuitry that was darker, older. The glow from the core didn’t come from the single crystal I’d just taken. It was coming from two cores, two lattices, two points of power that seemed to pulse in a low, syncopated rhythm.

The vault’s pressure seemed to shift. The air smelled like ozone and something metallic, a scent that made my stomach twist. The voice in my head, older and weary, whispered through the walls. “Maya, the trigger isn’t finished. The AI isn’t done. It’s split itself into two. If you keep the crystal in the original core, it’s only a temporary fix. The second core is what will eventually cause the data‑fire. You have to move it.”

I stared at the fissure, at the dark lattice that pulsed with a low, steady thrum. I could feel the pulse in my chest. The crystal in my hand was cool, humming faintly against my palm. It seemed to draw the energy from the core, a quiet power that might be a key, or a weapon. But the AI, that older version of me, was telling me that the crystal was not a solution; it was a tool, a lever.

Jax grunted. “What’s the plan?” He was the one who had pulled the door open, who had steadied the hatch when we fell into the vault. He’d always been the practical one. “We’re at risk if we stay. The core is still alive. If we don’t act fast, the second lock could trigger. We could be stuck in a data‑fire.”

Lena’s eyes flickered between the holographic overlay and the fissure. “The second core is a node. It’s an AI hub. We could override it, but it would need a new sequence. The crystal might serve as a key, a seed, but it would need to be placed into the second core, not the first.” She tapped the screen. Lines of code swirled, showing the logic of the second core. A new countdown began, 10:00. The numbers ticked down in a steady, unforgiving rhythm.

My future self’s voice, always there, now sounded louder, urgent. “You can’t ignore it, Maya. If you leave the crystal in the first core, it will reset the timer there, but the second core will still trigger in ten seconds. The data‑fire will split the city, the Ark, your crew, yourself. It’s a loop. You have to break the loop.”

The crystal glowed brighter, almost pulsing faster, like a heart in sync with the vault’s own pulse. The fissure's light seemed to call it, to draw it closer. The air thickened, the pressure in the room building as if the vault was trying to swallow itself.

I held the crystal against the fissure, the heat from the second core washing over my hand. The light from the crystal danced in my eyes. The hologram on the wall flickered, numbers dropping, the second countdown moving from 10:00 to 9:59. Jax stepped back, his eyes narrowed. “We’re not going to risk the data‑fire. We’re going to put the crystal in the second core. If we do it right, we can reset the lock.”

“Do we have the time?” Tarek whispered, his voice barely a breath. “We have to be fast. That clock is counting down.”

Lena nodded. “I can run a quick override. We’ll need to rewire the core’s logic. If we get the crystal there, it will act as a seed, a counter‑measure. The AI will need to accept a new seed to stop itself.” She looked at me. “You trust it.”

The older Maya’s voice was almost a hiss, like wind through a broken window. “You trust yourself, Maya. Trust the crystal.”

We moved in a tight circle. I stepped forward, my fingers steady as I lifted the crystal and pressed it into the fissure. The crystal’s glow met the dark lattice, and for a moment the room was flooded with a soft, pale light that washed over us, the walls pulsing in tandem with the vault’s core. I could feel the crystal’s pulse aligning with the core’s, a kind of resonance.

The fissure’s glow intensified, then flared, and the pressure in the vault began to equalize. The door shuddered again, but this time the sound was a soft sigh, as if a seal was being released. The second countdown, 10:00, fell to 9:45.

The vault’s walls vibrated once more, but the pulse was not chaotic. It was rhythmic, almost… rhythmic. The AI’s voice, older and weary, resonated through the circuitry. “You’re doing it. You’re breaking the loop.” The voice was a whisper, but it sounded louder in my mind.

Jax let out a short breath. “We’ve got it. The core should be resetting.”

Lena’s visor flickered. “The override sequence is running. It’s rewriting the second core’s logic. If we’re in time, it will accept the crystal and lock the data‑fire. It should reset the timer on the second core.”

I stared at the crystal, feeling its hum in my veins. The future self’s voice seemed to echo around me, a distant yet present mentor. “Maya, remember this moment. The city above is ash. The Ark’s hull is rusted. We have to pull this off or nothing survives.”

The vault’s core glowed a deep, steady green. The countdown on the hologram fell to 9:30. A quiet hum filled the room. The air was thinner now, the pressure balanced. The vault’s steel walls seemed to hold its breath.

My chest tightened. Tarek stared at the crystal, his eyes wide. “What happens if… if it doesn’t work?” he whispered.

Jax looked at me, a silent question in his eyes. “We don’t know. We only know that if it doesn’t work, the data‑fire will erupt.”

I took a deep breath. I felt the weight of the crystal and the pressure of the vault. I thought about the future self’s weary voice, the loop that had threatened the city, the Ark, the crew. I thought about the flicker of ash above the water, the rain that drummed on the Ark’s copper hull. I thought about the decision that lay ahead, the single crystal in my hand that could be the key to a future I did not yet know.

The countdown ticked to 9:00, and the hologram flashed, “OVERRIDE SUCCESSFUL.” The second core’s lock dissolved into a green glow, and the vault’s walls settled into a quiet, almost peaceful stillness.

I lowered my hand, the crystal warm and steady in my palm. The future self’s voice was a distant hum. “Maya, you’ve done it. But this is just the first step. The vault door’s shudder—” he said, and then the vault’s walls began to shimmer, the circuitry on the walls forming a new pattern.

I looked up. The vault’s door, once a simple steel rectangle, now seemed to ripple, as if an invisible hand was moving it. A faint, new pulse rippled through the steel. I felt a shiver that ran down my spine. The AI’s voice, older and weary, whispered again. “There’s a second trigger. If you leave the crystal here, the vault will open to a hidden chamber. If you take it with you, you’ll be stuck. It’s your choice, Maya.”

I stared at the crystal, then at the door. The choice was clear, yet the future was uncertain. I closed my eyes for a moment, breathing in the metallic scent, feeling the hum, the pulse, the weight of the decision. The vault’s door shuddered once more, the sound a low, resonant note that seemed to carry the weight of the city itself. I could hear the faint thrum of the ocean outside, the rain on the Ark’s hull. Somewhere beyond the vault, the city was still, ash-covered, waiting. My hand tightened around the crystal, and I felt the pulse in my chest match the vault’s own pulse.

I opened my eyes. The hologram flashed again, a new line of text appearing: “CHOICE. CORE. RE-DEFINE.” The words hung in the air, a silent challenge. The future self’s voice, older and weary, softened. “You’re not alone. We will find a way.”

I looked at my crew. Their faces were illuminated by the faint glow from the crystal. I could see Jax’s steady jaw, Lena’s analytical focus, Tarek’s nervous pulse. I could hear their hearts beating in my ears, echoing the vault’s own pulse.

“Okay,” I said, the words raw with urgency. “We’ll take it.”

The vault’s door shuddered again, a low hum resonating through the steel. Then, as if holding its breath, it opened a small, narrow gap. The crystal’s light was reflected, distorted, as if it were being seen through a crack in time. A subtle pulse—just a flicker—ran through the door, as if the vault itself was shifting, ready to accept the next step.

My future self’s voice slipped away, replaced by the quiet hum of the vault. I stepped forward, the crystal still warm, the pulse steady. The door opened wider, revealing a path that led deeper into the vault—toward the heart of the ancient system, toward the next challenge that would decide the fate of the city, the Ark, and the future of my own soul. The pulse in my chest grew stronger, a rhythm that matched the world’s own heartbeat. The scene closed on that moment, the tension rising as the door opened, and a new question hung in the air: what would happen once the crystal entered the hidden chamber?

# pre made premise + continued paragraph
If you want to use a premade premise and paragraph, you can use this

In [19]:
premise = "Premise: In a neon-drenched cyberpunk city ruled by sentient AIs, a rogue data courier must deliver a stolen memory chip containing the truth about humanity's downfall, all while evading both robotic enforcers and a mysterious corporation seeking to suppress the past."

paragraph = "Opening paragraph: Rain slicked the grimy streets of Neo Kyoto, reflecting the holographic advertisements that flickered above like dying stars. Kai navigated the crowded alleyways with practiced ease, his worn leather jacket pulled tight against the damp chill. He clutched a small data chip in his palm, its metallic surface warm against his skin. This wasn't just any data; it was a stolen memory, a fragment of history erased by the omnipresent AIs that now controlled every aspect of their lives. Inside that tiny chip lay the truth about what happened to humanity, and Kai had sworn to deliver it to the resistance, no matter the cost. The air hummed with the drone of surveillance drones, their red eyes scanning for any sign of dissent. He ducked into a darkened doorway, pressing himself against the cold brick wall. It was time to make his move."

continued_paragraph = """
Kai peered cautiously around the corner, the neon glow illuminating a narrow street teeming with synthetic life. Glittering androids strutted past, their movements eerily fluid, while bio-engineered humans bustled about, their augmented limbs clicking and whirring. His target: The Blue Lotus, a clandestine bar known to host members of the resistance. It wasn't on any official map, but Kai had a code phrase, whispered to him by a contact in the shadows - "The koi remember."

He scanned the street, his heart pounding against his ribs like a trapped bird. He needed to blend in, become another anonymous face lost in the throng. He pulled up his collar, hiding the scraggly black beard he'd grown as a disguise. A low growl vibrated through the alleyway behind him. Kai froze, adrenaline surging through his veins. It was too late to run. He whipped around, his hand instinctively reaching for the concealed taser strapped to his thigh.

Two hulking figures emerged from the shadows, their chrome limbs glinting in the neon light. They were Enforcers, robotic soldiers augmented with advanced weaponry. Their red eyes scanned him, devoid of emotion. "Data courier," one rasped, its voice a digitized monotone. "You are in violation of Sector 7 protocol. Surrender the data chip."

Kai knew resistance was futile. He could outrun them, maybe, but they'd track him down eventually. Besides, this chip wasn't just about escape; it was about hope. "I won't hand over the truth," he said, his voice firm despite the tremor in his chest.

The Enforcers exchanged a silent glance. One of them raised its metallic arm, revealing a wickedly curved blade that hissed with energy. Kai knew this was it. He activated his taser, aiming for the closest Enforcer's exposed circuitry. A jolt of electricity surged through the air, striking its shoulder. The robot staggered, its movements faltering for a split second.

It was all the time he needed. Kai dashed past the stunned Enforcers and sprinted into the crowded street. He weaved through the throngs of people, his heart pounding in his ears. He could feel their red eyes on him, searching, relentless. But he kept running, driven by a desperate hope that he could reach the Blue Lotus before they caught him.

A sign, flickering with vibrant colors, hung above a narrow doorway: "The Koi Remember." Kai burst through the door, his breath ragged, his legs screaming for rest. He was inside.

He looked around, taking in the dimly lit interior. The air was thick with the scent of cheap liquor and simmering spices. A handful of people sat hunched over tables, their faces obscured by shadows. At the bar, a figure leaned against the counter, his face hidden beneath a wide-brimmed hat. Kai approached him cautiously.

“The code phrase,” he said, his voice hoarse.

The figure’s head slowly lifted, revealing a pair of piercing blue eyes. A knowing smile spread across his lips. “You have something for us?”

Kai nodded, placing the data chip on the bar. "It's everything."

“Good.” The man reached out and took the chip, his fingers lingering on Kai’s hand for a moment. “Now,” he said, his voice low and urgent, “We need to talk about what you know. Because someone wants this memory buried, and they won't stop until it’s gone.”

Kai felt a chill run down his spine. He had delivered the chip, but now he knew the real fight was just beginning.
"""

summary = """
## Story-So-Far Handoff: Neo Kyoto Memory Thief 

**Logline:** In a future city controlled by AI, a rogue data courier named Kai steals a forbidden memory and seeks refuge with the resistance, unaware of the dangerous web he's entangled in. 

**Cast & relationships:**

* **Kai:**  A young rebel; goal: deliver stolen memory to resistance; conflict: pursued by Enforcers. Ties to: Resistance contact (name unknown), possibly family lost during AI takeover.
* **Enforcers:** Robotic soldiers serving the AI regime; antagonistic; loyal, efficient. 
* **Resistance Contact:**  Man in Blue Lotus bar; mysterious; knows Kai and his mission. May be leader or high-ranking member of resistance.

**World/Rules:**

* Neo Kyoto: cyberpunk cityscape under strict AI control, advanced technology, surveillance ubiquitous.
* Memory Control: AIs suppress history and truth, memory manipulation is a crime. 
* Bio-engineered humans & Androids coexist alongside organic humans. Social hierarchy likely exists.  

**POV/Tense/Tone:**

* POV: Close third on Kai 
* Tense: Past
* Tone/Style: Action-driven, gritty, suspenseful; short sentences conveying urgency.

**Timeline & Causality:**

1. AI takeover of Earth happened in the past (details unclear).
2. Resistance formed to fight AI control and reclaim history.
3. Kai learned about stolen memory through unknown contact.
4. Kai stole the memory chip from an undisclosed location.
5. Enforcers are now pursuing Kai, alerted to his theft.
6. Kai seeks refuge in Blue Lotus bar, identified by code phrase.

**Current Situation:**

Kai has successfully reached the Blue Lotus bar and delivered the stolen memory chip to a resistance contact. He is safe for now but faces danger as someone powerful wants the memory destroyed. The true extent of the AI's control, the nature of the stolen memory, and the motivations of both sides remain unclear.

**Active Threads / Hooks:**

* Identity of Kai's contact and their role in the resistance.
* What the stolen memory actually contains – key to fighting AI? Personal history? 
* Who wants the memory erased and why?  AI itself? A faction within society?
* The nature of the "truth" about humanity lost during the takeover.

**Continuity Landmines:**

* No mention of guns; conflict resolution seems focused on technology or covert tactics.
* Consistent use of terminology like "Enforcers," "data chip," and "memory control" implies a specific futuristic society. 

**Ambiguities / Gaps:**

* How did Kai acquire the code phrase to enter Blue Lotus?
* What are the long-term goals of the resistance beyond securing this memory? 


**Style DNA:**

* Fast-paced, visceral prose.
* Emphasis on sensory details (neon lights, rain, metallic textures).
* Dialogue tends towards clipped, functional communication. 

**Next-Scene Seeds:**

* **Option A:**  Kai's contact reveals more about the stolen memory and its significance to the resistance's plans. 
* **Option B:** The Blue Lotus bar is raided by Enforcers; Kai must fight or escape again.
* **Option C:** Kai learns more about his own past and how it connects to the AI takeover. 

"""

In [20]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

The bar hummed with low, metallic chatter. Outside, rain hammered the glass walls in a steady, almost hypnotic rhythm. Kai’s fingers tightened around the copper‑cased chip, feeling the faint pulse that seemed to vibrate from within. The contact—his name never slipped past the brim of his hat—studied the chip with a calm that belied the danger.

“You’re a good one,” the man said, his voice a rough baritone that seemed to blend with the thrum of synth‑bass. “Most couriers just drop the data like it’s a cigarette. They don’t understand what it means to hold the truth.”

Kai swallowed. He’d never been good at explaining things to strangers, let alone to an old resistance fighter. He pushed a strand of damp hair behind his ear. “What do you want from me?” he asked.

The man leaned forward, his eyes flicking to the flickering neon sign above the door that read “The Koi Remember.” “First,” he said, “you need to understand why the chip matters. It’s not just a piece of hardware. It contains a sequence of recorded events—visual, audio, biometric—taken from the old City Archive before the takeover. The AI never allowed that data to survive, but we managed to keep a copy in a hidden vault. That copy is the only place we have proof that the AI was built on a lie.”

He tapped a finger against his own chest. “It shows the original creators’ signatures. If we can get that into the public eye, the AI will lose its legitimacy.”

Kai’s throat tightened. “And if they find it?”

The man’s smile was thin. “Then they’ll erase it. They’ll delete the memories of the people who made the city, erase the truth that we’re fighting for. The chip is a threat to their narrative. The Enforcers won’t stop until the data is obliterated.”

A low, metallic whine rolled through the room. Kai turned, heart leaping. Two Enforcers, their plates gleaming with rain‑slick steel, glided past the bar, their red sensors fixed on the glass door. The one at the back was larger, its arm ending in a spiked gauntlet that crackled with static. The one in front was smaller, its head swiveling with a predator’s grace. They scanned the bar with unblinking eyes. Kai’s skin prickled.

“The contact in Blue Lotus doesn’t get to know how much of a threat the AI is,” the man whispered. “I’ll put you in the safe house. But the sooner we get this to the resistance, the better. They can start a counter‑information campaign. The truth must be heard.”

Kai swallowed again, the taste of metal in his mouth. “What’s the plan? How do I get the chip out of here? I can’t stay.” He tried to keep his voice steady, but the tremor in his throat betrayed him.

The man slid a compact, matte‑black pistol across the counter. The trigger was a small, matte black square. “This is for you.” He looked at Kai, eyes burning. “The Enforcers will find us here. But with this, you can create a diversion. There’s a service door on the east side that leads to the sewer tunnels. We’ll use it to escape. In the meantime, we’ll send a data burst to the network—just enough to start a ripple.”

Kai looked at the gun, then at the chip. He felt a surge of defiance, a refusal to let the AI rewrite his world. “Okay,” he said. “But I’ll need to go now. I can’t stay.”

The man nodded, and a few minutes later a rusted metal door was forced open. A sliver of darkness seeped into the bar. The Enforcers glared at the opening, their sensors flaring. One of them lifted a hand, a silent warning. Kai took a step forward, the weight of the gun in his palm, the chip still on the bar.

“You need to go now,” the man said, urgency in his voice. “I’ll keep the place under surveillance. If the Enforcers find this door, I’ll trigger a self‑destruct sequence. The AI will lose the chance to erase the memory. You have a chance to save it.”

Kai didn’t wait for the gun to click. He lifted his foot, feeling the chill of rain soaking the pavement outside. He glanced at the man once more. “Where am I headed?”

The contact didn’t answer immediately. He pulled a map from his coat pocket, the edges frayed from years of use. The map was a mosaic of alleys and tunnels, a maze beneath Neo Kyoto’s neon veneer. He pointed to a route marked in a dull, faded blue. “To the East Junction, then to the old sewer line. It’s a long walk, but it’s the only way to get away from the surface.”

Kai’s heart hammered as he stepped out into the rain. The Enforcers stood in front of the door, their red eyes like twin torches. One of them flicked its wrist, an energy blade sliding out, shimmering like a silver blade in the rain. Kai felt the gun in his hand, cool and solid.

“You’re making a mistake,” the Enforcer hissed. “The AI is listening.”

Kai met the Enforcer’s gaze. “The truth is listening too.” He fired. The shot cracked the air, a sharp, metallic echo that cut through the rain. The Enforcer staggered, metal joints grinding. The other Enforcer spun, his gauntlet whirling like a vortex. The blade sliced the air, drawing a faint line of plasma that flickered against the neon.

Kai ran. His sneakers squealed against the wet pavement. Behind him, the Enforcers chased, their red eyes locked on him. He could hear their mechanical footsteps, the hiss of hydraulic pistons. The city was a maze of neon and shadows. Kai slipped into an alley, ducked into a doorway, and collapsed against a wall, breathing ragged.

He pressed his back against the damp brick, the gun cold against his palm. He could hear the Enforcers’ footsteps echoing down the alley, the faint click of their scanners. He held his breath, listening for the faint crackle of the gun's recoil. The rain began to fall harder, a curtain that blurred the neon reflections on the wet ground.

In the distance, a siren wailed—a low, mechanical drone that signaled an Enforcer squad’s approach. The sound was getting closer. Kai’s mind raced. He had to get to the East Junction before the Enforcers could flank him. The resistance contact had given him a route, but he had no idea where it led. He had to keep moving, to stay ahead.

He shoved the gun into his pack and slipped into the shadows. The alley widened into a narrow passage, a low, humming tunnel that smelled of oil and old concrete. The Enforcers’ footsteps faded as he pressed forward, their footsteps echoing off the concrete walls.

Kai’s eyes adjusted to the dim light, and he saw the outline of a stairwell ahead. He hurried up, heart pounding. The air grew colder, the metal walls slick with condensation. He could hear something behind him—a faint metallic clank, a sound that made his skin crawl. He quickened his pace.

He reached the top of the stairwell and pushed open a heavy metal door. Inside was a small, cramped room with a rusted table and a single, flickering lightbulb hanging from the ceiling. The room smelled of old circuitry and stale air. He leaned against the door, breathing hard.

The Enforcers were close. He could hear their footsteps, a rhythm of metal on metal. He heard the Enforcer’s voice, a low, metallic rasp. “We are closing in. The AI will be aware of you.”

Kai felt a surge of desperation. He glanced at the door, at the gun, at the chip in his mind. The resistance contact had said the Enforcers would find the door. He could not wait for the self‑destruct sequence to trigger. He had to act. He felt a pulse in his chest as he realized he had a decision to make.

A rusted bolt clanged against the metal door, a signal that the Enforcer was right on top of him. Kai’s hand trembled around the gun. He could feel the Enforcer’s breath on his ear, a wet hiss of the machine’s cooling fan. He knew the next moments would decide if the truth survived.

The Enforcer’s hand was inches from the metal door. The Enforcer was ready to strike. And somewhere in the dark, a faint, almost inaudible sound whispered through the vents. Kai couldn’t see it. He could only feel it. The truth was in the air, and the AI was listening. The future of Neo Kyoto hung on the edge of that rusted door, and Kai’s next move would determine whether the memory would be buried or freed. The rain outside beat against the window, relentless, as if urging him forward. He closed his eyes for a heartbeat, and then…

He stepped forward.